In [ ]:
# Ok, you gotta redo the filtering strategy, it's wrong to filter by pairs, this way gene pairs may be isolated (of course)

# e.g.,

# 2955  HF571520.1_1897__HF571520.1_1897_1878235_18794...  HF571520.1_1897  HF571520.1     1897  (1878235, 1879485)    pos
# The above was hit of 899.1
# 1271  HF571520.1_1898__HF571520.1_1898_1879706_18817...  HF571520.1_1898  HF571520.1     1898  (1879706, 1881700)    neg
# This one hit of 171.1 and 170.1


# 899: 1897 (pos)
# 171: 411, 1898
# 170: 410, 1898


# Solutions:

"""
1. Sort hmmer hits by contig name, perhaps use a dictionary. Perhaps add tag to label containing hmm name
2. Search for hits within the same contig that satisfy the structure
"""

In [1]:
import pandas as pd
from pynteny.utils import readFromPickleFile
from pynteny.filter import LabelParser


hmm_hits = readFromPickleFile("/home/robaina/Documents/Pynteny/hmm_hits.pickle")

hmm_order = ["TIGR00899.1", "TIGR00171.1", "TIGR00170.1", "TIGR00973.1"]
n_hmms = len(hmm_order)
hmm_order_dict = dict(zip(hmm_order, range(n_hmms)))

hit_labels = {}
labelparser = LabelParser()
for hmm, hits in hmm_hits.items():
    labels = hits.id.values.tolist()
    if not labels:
        raise ValueError(
            f'No records found in database matching HMM: {hmm}'
            )
    hit_labels[hmm] = labelparser.parse_from_list(labels)
    hit_labels[hmm]["hmm"] = hmm



# Create single dataframe with new column corresponding to HMM and all hits
all_hit_labels = pd.concat(hit_labels.values()).groupby("contig").filter(lambda x : len(x) >= n_hmms).sort_values(["contig", "gene_pos"])  # remove contigs with less hits than the number of hmms in synteny structure
all_hit_labels["gene_pos_diff"] = all_hit_labels.gene_pos.diff()

# Drop sequences hit by more than one hmm
all_hit_labels = all_hit_labels.drop_duplicates(subset=all_hit_labels.columns.difference(["hmm", "gene_pos_diff"]), keep=False)

# Reset index
all_hit_labels.reset_index(drop=True, inplace=True)

# Add HMM codes
all_hit_labels["hmm_code"] = all_hit_labels.hmm.apply(lambda hmm: hmm_order_dict[hmm])


# Strand in integers
all_hit_labels["strand"] = all_hit_labels.strand.apply(lambda strand: -1 if strand == "neg" else 1)

contig_names = all_hit_labels.contig.unique()

In [2]:
hmm_order_pattern = ["TIGR00899.1", "TIGR00171.1"]
hmm_code_order_pattern = [0, 1] # "TIGR00899.1", "TIGR00171.1" # list(hmm_order_dict.values())
strand_order_pattern = [0, 0] #,0, 0]
distance_order_pattern = [100] #, 0, 1]


def contains_hmm_pattern(data: pd.Series) -> int:
    return 1 if data.values.tolist() == hmm_code_order_pattern else 0

def contains_distance_pattern(data: pd.Series) -> int:
    return 1 if all(
        [   (data_dist <= pattern_dist and data_dist > 0)
            for data_dist, pattern_dist in zip(data.values.tolist()[1:], distance_order_pattern)
            ]
        ) else 0

def contains_strand_pattern(data: pd.Series) -> int:
    strand_comparisons = []
    for data_strand, pattern_strand in zip(data.values, strand_order_pattern):
        if pattern_strand != 0:
            strand_comparisons.append(data_strand == pattern_strand)
        else:
            strand_comparisons.append(True)
    return 1 if all(strand_comparisons) == True else 0

In [3]:
from pynteny.filter import LabelParser

# hmm_order = ["TIGR00171.1", "TIGR00170.1"] #["TIGR00899.1", "TIGR00171.1", "TIGR00170.1", "TIGR00973.1"]
# hmm_order_dict = dict(zip(hmm_order, [1, 2]))

output_file = "results.tsv"

n_hmms = len(hmm_order_pattern)
output_lines = []

for contig in contig_names:

    synteny_hits = []
    contig_hits = all_hit_labels[all_hit_labels.contig == contig].reset_index(drop=True)

    if set(hmm_order_pattern) == set(contig_hits.hmm.unique()):
        
        hmm_match = contig_hits.hmm_code.rolling(window=n_hmms).apply(contains_hmm_pattern)
        strand_match = contig_hits.strand.rolling(window=n_hmms).apply(contains_strand_pattern)
        distance_match = contig_hits.gene_pos_diff.rolling(window=n_hmms).apply(contains_distance_pattern)

        matched_rows = contig_hits[
            (hmm_match == 1) &
            (strand_match == 1) &
            (distance_match == 1)
        ]
        
        for i, row in matched_rows.iterrows():
            matched_hits = contig_hits.iloc[i - (n_hmms -1): i + 1]
            for label, hmm in zip(matched_hits.full.values, matched_hits.hmm):
                parsed_label = LabelParser.parse(label)
                synteny_hits.append(
                     (
                        f"{parsed_label['contig']}\t{parsed_label['gene_id']}\t"
                        f"{parsed_label['gene_pos']}\t{parsed_label['locus_pos']}\t"
                        f"{parsed_label['strand']}\t{hmm}\t{parsed_label['full']}\n"
                        )
                )
            output_lines.extend(synteny_hits)



with open(output_file, "w") as outfile:
    outfile.write("contig\tgene_id\tgene_number\tlocus\tstrand\tHMM\tfull_label\n")
    outfile.writelines(output_lines)

In [3]:
record_name = "CP000435.1_1 # 174 # 1331 # 1 # ID=1_1;partial=00;start_type=ATG;rbs_motif=None;rbs_spacer=None;gc_cont=0.567"


name_list = record_name.split(" ")
contig = "_".join(name_list[0].split("_")[:-1])
gene_number = name_list[0].split("_")[-1]
start, end = name_list[2], name_list[4]
strand = "pos" if name_list[6] == "1" else "neg"


header = f">{contig}_{gene_number}__{contig}_{gene_number}_{start}_{end}_{strand}"

print(header)

>CP000435.1_1__CP000435.1_1_174_1331_pos


In [1]:
import os 

from pynteny.utils import fullPathListDir

for file in fullPathListDir("/home/robaina/Databases/MarRef_1.7/prodigal"):
    if '.gbk' in file:
        os.remove(file)

In [4]:
from pynteny.preprocessing import parseProdigalOutput, mergeFASTAs


# mergeFASTAs(
#     "/home/robaina/Databases/MarRef_1.7/prodigal",
#     output_fasta="/home/robaina/Databases/MarRef_1.7/marref_prodigal.faa"
# )

parseProdigalOutput(
    prodigal_faa="/home/robaina/Databases/MarRef_1.7/marref_prodigal.faa",
    output_file="/home/robaina/Databases/MarRef_1.7/marref_prodigal_longlabels.faa"
)

In [1]:
from Bio import SeqIO, SeqRecord


file = "/home/robaina/Documents/Pynteny/MG1655.gb"
file = "/home/robaina/Databases/MarRef_1.7/prodigal/marref.gbk"
file = "/home/robaina/Documents/Pynteny/BACL149.gbk"
gbk_contigs = list(SeqIO.parse(file, 'genbank'))

In [2]:
gbk_contig = gbk_contigs[0]

[SeqRecord(seq=Seq('ATCAACACCCAGTTTTTTCCAAGGTAAGTTTTCTGGATCGCTTTCGTGAAAACA...GCC'), id='1', name='1', description='Genus species strain strain', dbxrefs=[])]

In [1]:
from pynteny.preprocessing import assignGeneLocationToRecords



gbk_file = "/home/robaina/Documents/Pynteny/MG1655.gb"

assignGeneLocationToRecords(
    gbk_file=gbk_file,
    output_fasta=None,
    nucleotide=False
)

In [ ]:
from pynteny.wrappers import runProdigal

# Run prodigal on assembly to obtain gene translation and positional info

runProdigal(
    input_file="genome_assembly.fasta",
    output_prefix=None,
    output_dir=None
)

In [2]:
# In case of split genomes, each part comes in a different gbk file,this would happen with differnt contigs as well.
# So, basically, I only need the gbk file, which can be obtanined from prokka with the genome assembly in fasta format.
# Genes appear to be sorted by loci in gbk files
# Actually, the departing point for a hmmer search with synteny info would be a fasta file containing reference sequences...
# so, we would need to obtain these sequences from the gbk file or map them somehow?
# Perhaps better to store positional info in a dict, this way no need to alter contig/sequence names

from Bio import SeqIO, SeqRecord


file = "/home/robaina/Documents/Pynteny/MG1655.gb"


gbk_contigs = list(SeqIO.parse(gbk_file, 'genbank'))

with open("LongLabels.fasta", "w") as outfile:

    for gbk_contig in gbk_contigs:
        
        gene_counter = 0
        for feature in gbk_contig.features:

            if "cds" in feature.type.lower():
                
                name = feature.qualifiers["locus_tag"][0].replace('_', '.')
                start, end, strand = str(feature.location.start), str(feature.location.end), feature.location.strand
                start = start.replace(">", "").replace("<", "")
                end = end.replace(">", "").replace("<", "")
                strand_sense = "neg" if strand == -1 else "pos"

                header = f">{name}__{gbk_contig.name.replace('_', '')}_{gene_counter}_{start}_{end}_{strand_sense}\n"
                if "translation" in feature.qualifiers:
                    translation = feature.qualifiers["translation"][0]

                    header = f">{name}__{gbk_contig.name.replace('_', '')}_{gene_counter}_{start}_{end}_{strand_sense}\n"
                    outfile.write(header)
                    outfile.write(translation + "\n")

                    gene_counter += 1